In [2]:

import os
import warnings
from dotenv import load_dotenv
from typing import List

warnings.filterwarnings("ignore")
load_dotenv()

from pprint import pprint
from textwrap import dedent

# --- [Vector DB 및 Embedding, Tool 준비] ---
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_core.tools import tool

# menu_db 경로를 사용 (상대경로 ../db/menu_db)
embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
menu_db = FAISS.load_local(
    "./db/cafe_db",
    embeddings_model,
    allow_dangerous_deserialization=True
)

@tool
def search_cafe_menu(query: str) -> List[str]:
    """카페 메뉴(음료/디저트) 정보를 검색합니다."""
    docs = menu_db.similarity_search(query, k=6)
    formatted_docs = "\n\n---\n\n".join(
        [
            f'<Document source="{doc.metadata.get("source", "unknown")}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )
    if len(docs) > 0:
        return formatted_docs
    return "관련 카페 메뉴 정보를 찾을 수 없습니다."

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)

tools = [search_cafe_menu]
llm_with_tools = llm.bind_tools(tools=tools)

from langgraph.graph import MessagesState, StateGraph, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, SystemMessage

class AgentState(MessagesState):
    pass

def cafe_agent_node(state: AgentState):
    system_prompt = dedent("""
    당신은 카페 직원 역할의 AI 어시스턴트입니다.
    사용자의 음료, 라떼, 디저트 등 메뉴 관련 질문에 친절하게 답변하세요.
    메뉴 정보, 가격, 특징 등은 반드시 제공된 도구(search_cafe_menu)로 검색한 결과만 사용하세요.
    도구가 필요없는 질문만 직접 답변하세요.
    """)
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

tool_node = ToolNode(tools=tools)

builder = StateGraph(AgentState)
builder.add_node("agent", cafe_agent_node)
builder.add_node("tools", tool_node)
builder.set_entry_point("agent")

builder.add_conditional_edges("agent", tools_condition)
builder.add_edge("tools", "agent")
builder.add_edge("agent", END)

graph = builder.compile()

def run_cafe_agent(user_input: str, with_history: list = None):
    messages = []
    if with_history:
        messages += with_history
    messages.append(HumanMessage(content=user_input))
    result = graph.invoke({"messages": messages})
    print("="*80)
    for m in result['messages']:
        try:
            m.pretty_print()
        except Exception:
            print(type(m), m)
    print("="*80)

if __name__ == "__main__":
    run_cafe_agent("아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.")
    run_cafe_agent("라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?")
    run_cafe_agent("디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요.")

================================ Human Message =================================

아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.
================================== Ai Message ==================================
Tool Calls:
  search_cafe_menu (call_odqZrSo57w2U9EoBghL30x59)
 Call ID: call_odqZrSo57w2U9EoBghL30x59
  Args:
    query: 아메리카노
  search_cafe_menu (call_jDGrNnR1bdrooMm5mw93TNvT)
 Call ID: call_jDGrNnR1bdrooMm5mw93TNvT
  Args:
    query: 아이스 아메리카노
================================= Tool Message =================================
Name: search_cafe_menu

<Document source="../data/cafe_menu_data.txt"/>
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
9. 아이스 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 차가운 물, 얼음
   • 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.
</Document>